In [2]:
import requests
import pandas as pd

/Users/chaddiao/Desktop/Python Projects/OpenFDA API/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
df = pd.DataFrame(pd.read_csv("FDA_recalls_20-24.csv"))
df.head()

,FEI Number,Recalling Firm Name,Product Type,Product Classification,Status,Distribution Pattern,Recalling Firm City,Recalling Firm State,Recalling Firm Country,Center Classification Date,Reason for Recall,Product Description,Event ID,Event Classification,Product ID,Center,Recall Details
0,3004497354,GE Healthcare GmbH,Devices,Class II,Ongoing,US Nationwide distribution including in the st...,Munich,-,Germany,09/13/2024,The National Institute of Standards and Techno...,"ViewPoint; system, imaging processing, radiolo...",95167,Class II,209643,CDRH,https://www.accessdata.fda.gov/scripts/ires/?P...
1,3004497354,GE Healthcare GmbH,Devices,Class II,Ongoing,US Nationwide distribution including in the st...,Munich,-,Germany,09/13/2024,The National Institute of Standards and Techno...,"ViewPoint 6, Model Numbers: a) H47581RJ, b) H...",95167,Class II,209644,CDRH,https://www.accessdata.fda.gov/scripts/ires/?P...
2,2523676,"B. Braun Medical, Inc.",Devices,Class II,Ongoing,US Nationwide distribution.,Allentown,Pennsylvania,United States,09/13/2024,Potential for product leakage from the Spin Lo...,AET36 ANESTH EXT SET - Anesthesia Extension Se...,95153,Class II,209610,CDRH,https://www.accessdata.fda.gov/scripts/ires/?P...
3,3002095335,Boston Scientific Corporation,Devices,Class II,Ongoing,"US, Colombia, Philippines, Russia",Maple Grove,Minnesota,United States,09/13/2024,There is the potential for holes in the clear ...,"Flexima APDL Drainage Catheter System and Kit,...",95186,Class II,209691,CDRH,https://www.accessdata.fda.gov/scripts/ires/?P...
4,3002095335,Boston Scientific Corporation,Devices,Class II,Ongoing,"US, Colombia, Philippines, Russia",Maple Grove,Minnesota,United States,09/13/2024,There is the potential for holes in the clear ...,Flexima Regular Kit Biliary Catheter System Ki...,95186,Class II,209692,CDRH,https://www.accessdata.fda.gov/scripts/ires/?P...


In [3]:
print(df.columns)

Index(['FEI Number', 'Recalling Firm Name', 'Product Type',
       'Product Classification', 'Status', 'Distribution Pattern',
       'Recalling Firm City', 'Recalling Firm State', 'Recalling Firm Country',
       'Center Classification Date', 'Reason for Recall',
       'Product Description', 'Event ID', 'Event Classification', 'Product ID',
       'Center', 'Recall Details'],
      dtype='object')


In [153]:
# these are the most severe recalls
ClassI_df = df[df['Event Classification'] == 'Class I']
ClassI_df.head()

,FEI Number,Recalling Firm Name,Product Type,Product Classification,Status,Distribution Pattern,Recalling Firm City,Recalling Firm State,Recalling Firm Country,Center Classification Date,Reason for Recall,Product Description,Event ID,Event Classification,Product ID,Center,Recall Details
24,3012307300,Smiths Medical ASD Inc.,Devices,Class I,Ongoing,Worldwide,Minneapolis,Minnesota,United States,09/11/2024,There is the potential for a disconnection of ...,"smiths medical portex BLUselect, tracheostomy ...",95016,Class I,209111,CDRH,https://www.accessdata.fda.gov/scripts/ires/?P...
25,3012307300,Smiths Medical ASD Inc.,Devices,Class I,Ongoing,Worldwide,Minneapolis,Minnesota,United States,09/11/2024,There is the potential for a disconnection of ...,smiths medical portex BLUgriggs Percutaneous D...,95016,Class I,209113,CDRH,https://www.accessdata.fda.gov/scripts/ires/?P...
26,3012307300,Smiths Medical ASD Inc.,Devices,Class I,Ongoing,Worldwide,Minneapolis,Minnesota,United States,09/11/2024,There is the potential for a disconnection of ...,smiths medical portex BLUperc Percutaneous Dil...,95016,Class I,209114,CDRH,https://www.accessdata.fda.gov/scripts/ires/?P...
27,3012307300,Smiths Medical ASD Inc.,Devices,Class I,Ongoing,Worldwide,Minneapolis,Minnesota,United States,09/11/2024,There is a potential for inadvertent tidally v...,"Pneupac paraPAC plus 300 ventilator kit, REF P...",95025,Class I,209122,CDRH,https://www.accessdata.fda.gov/scripts/ires/?P...
28,3012307300,Smiths Medical ASD Inc.,Devices,Class I,Ongoing,Worldwide,Minneapolis,Minnesota,United States,09/11/2024,There is a potential for inadvertent tidally v...,Pneupac paraPAC plus 310 ventilator kit with P...,95025,Class I,209123,CDRH,https://www.accessdata.fda.gov/scripts/ires/?P...


In [ ]:
#see all unique values of reason for recall column
recall_reasons = ClassI_df['Reason for Recall'].unique().tolist()
recall_reasons

### SpaCy NLP

In [6]:
import spacy

# Load spaCy language model
nlp = spacy.load("en_core_web_md")

# Function to categorize text using spaCy similarity with multiple keywords
def spacy_categorize(text, exact_match_keywords, category_keywords, threshold=0.6):
    # first check for exact matches
    for keywords, category in exact_match_keywords.items():
        for keyword in keywords:
            if keyword in text.lower():
                #print(f'found exact match of "{keyword}" in the text')
                return category, 1.0
    
    doc = nlp(text)
    best_match = None
    highest_score = 0  

    # If no exact match, use similarity based categroization
    # Loop over each category and its associated list of keywords
    for keywords, category in category_keywords.items():
        for keyword in keywords:
            keyword_doc = nlp(keyword)
            similarity = doc.similarity(keyword_doc)  # Get similarity score
            if similarity > highest_score:
                highest_score = similarity
                best_match = category

    # Return the best category match and the similarity score
    if highest_score >= threshold:
        return best_match, highest_score
    else:
        return "Other", highest_score
    


### Recall Categorization

In [ ]:
exact_match_keywords = {
    ("manufacturing defect", "tear", "catheter", "connector"): "Manufacturing Defects",
    ("software update", "security issue", "manual update"): "Software Issues",
    ("design", "battery", "fracture"): "Design/Component Flaws",
    ("sterility", "contamination"): "Contamination or Sterility Issues",
    ("expire", "expiration"): "Expiry Issues",
    ("function", "functional", "malfunction", "operation", "operational"): "Operation or Functional Issues"

}

category_keywords = {
    ("disconnection", "manufacturing defect", "failure", "defect", "tear", "fault", "catheter", "detachment", "malformed"): "Manufacturing Defects",
    ("software update", "software version", "software issue", "security", "user manual"): "Software Issues",
    ("design flaw", "electromagnetic compatibility", "heating issues", "heat retention", "fracture", "improper ventilation", "overheat", "connector problem", "battery", "component failure", "compromise integrity"): "Design/Component Flaws",
    ("regulatory", "non-compliance" "safety standard violation"): "Compliance Violations",
    ("sterility", "contamination", "quality issues"): "Contamination or Sterility Issues",
    ("expiration date", "outdated components"): "Expiry Issues",
    ("operational failure", "incorrect functioning"): "Operation or Functional Issues"
}

# Sample use
text = "Reports of disconnection of the 15mm connector from the endotracheal tube (ET tube) for the affected products."
category, score = spacy_categorize(text, exact_match_keywords, category_keywords)
print(category, score)

Manufacturing Defects 1.0


In [230]:
recall_reasons = []
for text in ClassI_df["Reason for Recall"]:
    recall_reasons.append(text)
# note: this takes over a minute to run for just ClassI_df, which is about 3k rows.

results = []
for reason in recall_reasons:
    category, score = spacy_categorize(reason, exact_match_keywords, category_keywords)
    results.append({"Recall Reason": reason, "Category": category, "Score": score})

recall_reason_df = pd.DataFrame(results)
recall_reason_df.to_csv("categorized_recalls.csv", index=False)

In [224]:
recall_reason_df['Category'].value_counts()

Category
Design/Component Flaws               7016
Manufacturing Defects                2968
Software Issues                      1115
Contamination or Sterility Issues     957
Operation or Functional Issues        461
Other                                 316
Expiry Issues                         305
Operational or Functional Issues       47
Compliance Violations                  13
Name: count, dtype: int64

### Device Categorization

In [ ]:
device_exact_match_keywords = {
    # You need at least 2 items in the key (tuple) for this to work so I have to duplicate some
    ("surgical packs", "surgical kits", "medline"): "Surgery Procedure Kits",
    ("imaging", "radiology", "ultrasound"): "Imaging/Radiology Systems",
    ("anesthesia", "anesthesia"): "Anesthesia Equipment",
    ("catheter", "drain"): "Catheters",
    ("infusion", "infusion"): "Infusion Pumps",
    ("blood pump", "blood pumps"): "Blood Pumps",
    ("craniotomy", "craniotomy"): "Craniotomy Equipment",
    ("syringe", "syringe"): "Syringe Equipment",
    ("tracheostomy", "tracheostomy", "trach", "endotracheal tubes"): "Trachea Devices",
    ("ventilator", "respiratory", "Trilogy Evo", "Pneupac", "oxygen", "respironics"): "Ventilation and Respiratory Devices",
    ("monitoring", "monitoring"): "Monitoring Devices",
    ("neonatal incubator", "incubators"): "Incubators",
    ("cardiac", "balloon pump", "heart", "defibrillator", "defibrillators"): "Cardiac Assist Devices",
    ("covid", "covid-19"): "COVID-19 Testing Kits"

}

device_category_keywords = {
    ("surgical packs", "surgical kits", "trays"): "Surgery Procedure Kits",
    ("viewpoint", "imaging processing", "radiological"): "Imaging/Radiology Systems",
    ("anesthesia extension set"): "Anesthesia Equipment",
    ("drainage catheter", "locking pigtail", "drainage", "drain bag"): "Catheters",
    ("infusion pump", "monitoring system"): "Infusion Pumps",
    ("blood pump", "ventricular", "blood flow"): "Blood Pumps",
    ("craniotomy equipment"): "Craniotomy Equipment",
    ("syringe", "syringe", "insulin"): "Syringe Equipment",
    ("tracheostomy", "tube", "cuffed", "portex", "endotracheal", "airway"): "Trachea Devices",
    ("ventilator", "respiratory", "oxygen delivery", "oxygen reservoir", "mask", "resuscitator"): "Ventilation and Respiratory Devices",
    ("monitoring", "continuous monitoring"): "Monitoring Devices",
    ("neonatal incubator", "incubators"): "Incubators",
    ("cardiac", "cardiosave", "intra-aortic balloon pump", "heart", "defibrillators"): "Cardiac Assist Devices",
    ("covid", "covid-19 testing kits"): "COVID-19 Testing Kits"
}


# Sample use
text = "EcoGel 200 - RED Medical Supplies Clear Ultrasound Gel, Model No. 250UG-CMU"
category, score = spacy_categorize(text, device_exact_match_keywords, device_category_keywords)
print(category, score)


Imaging/Radiology Systems 1.0


In [231]:
prod_descs = []
for text in ClassI_df["Product Description"]:
    prod_descs.append(text)

# note: this takes 1-2 minutes to run for just ClassI_df, which is about 3k rows.
results = []
for desc in prod_descs:
    type, score = spacy_categorize(desc, device_exact_match_keywords, device_category_keywords)
    results.append({"Product Description": desc, "Type": type, "Score": score})
    
product_description_df = pd.DataFrame(results)
product_description_df.to_csv("categorized_devices.csv", index=False)

/var/folders/4n/lg_9zm91673b9xcs269mpwg00000gp/T/ipykernel_77589/858833609.py:24: UserWarning:

[W008] Evaluating Doc.similarity based on empty vectors.



In [232]:
product_description_df['Type'].value_counts()

Type
Other                                  3574
Blood Pumps                            2828
Anesthesia Equipment                   1567
Catheters                              1061
Imaging/Radiology Systems               622
Cardiac Assist Devices                  618
Ventilation and Respiratory Devices     600
Infusion Pumps                          570
Surgery Procedure Kits                  502
Monitoring Devices                      427
Syringe Equipment                       374
Trachea Devices                         260
COVID-19 Testing Kits                   130
Craniotomy Equipment                     60
Incubators                                5
Name: count, dtype: int64

## After categorizing, lets combine everything into one main dataframe.

In [234]:
# IMPORTANT: make sure the number of indexes for all new columns match the original dataframe
main_df = ClassI_df.reset_index(drop=True).copy()
main_df["Device Type"] = product_description_df["Type"]
main_df["Recall Category"] = recall_reason_df["Category"]

main_df.to_csv("categorized.csv", index=True)
main_df

,FEI Number,Recalling Firm Name,Product Type,Product Classification,Status,Distribution Pattern,Recalling Firm City,Recalling Firm State,Recalling Firm Country,Center Classification Date,Reason for Recall,Product Description,Event ID,Event Classification,Product ID,Center,Recall Details,Device Type,Recall Category
0,3004497354,GE Healthcare GmbH,Devices,Class II,Ongoing,US Nationwide distribution including in the st...,Munich,-,Germany,09/13/2024,The National Institute of Standards and Techno...,"ViewPoint; system, imaging processing, radiolo...",95167,Class II,209643,CDRH,https://www.accessdata.fda.gov/scripts/ires/?P...,Imaging/Radiology Systems,Design/Component Flaws
1,3004497354,GE Healthcare GmbH,Devices,Class II,Ongoing,US Nationwide distribution including in the st...,Munich,-,Germany,09/13/2024,The National Institute of Standards and Techno...,"ViewPoint 6, Model Numbers: a) H47581RJ, b) H...",95167,Class II,209644,CDRH,https://www.accessdata.fda.gov/scripts/ires/?P...,Imaging/Radiology Systems,Design/Component Flaws
2,2523676,"B. Braun Medical, Inc.",Devices,Class II,Ongoing,US Nationwide distribution.,Allentown,Pennsylvania,United States,09/13/2024,Potential for product leakage from the Spin Lo...,AET36 ANESTH EXT SET - Anesthesia Extension Se...,95153,Class II,209610,CDRH,https://www.accessdata.fda.gov/scripts/ires/?P...,Anesthesia Equipment,Manufacturing Defects
3,3002095335,Boston Scientific Corporation,Devices,Class II,Ongoing,"US, Colombia, Philippines, Russia",Maple Grove,Minnesota,United States,09/13/2024,There is the potential for holes in the clear ...,"Flexima APDL Drainage Catheter System and Kit,...",95186,Class II,209691,CDRH,https://www.accessdata.fda.gov/scripts/ires/?P...,Catheters,Contamination or Sterility Issues
4,3002095335,Boston Scientific Corporation,Devices,Class II,Ongoing,"US, Colombia, Philippines, Russia",Maple Grove,Minnesota,United States,09/13/2024,There is the potential for holes in the clear ...,Flexima Regular Kit Biliary Catheter System Ki...,95186,Class II,209692,CDRH,https://www.accessdata.fda.gov/scripts/ires/?P...,Catheters,Contamination or Sterility Issues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13193,3000209941,Inter-Med Llc,Devices,Class II,Terminated,"US, South Africa, Switzerland, Poland, Italy, ...",Racine,Wisconsin,United States,10/01/2019,The affected lots may be packaged in syringes ...,Vista Dental Products Sodium Hypochlorite - 6%...,83435,Class II,175277,CDRH,https://www.accessdata.fda.gov/scripts/ires/?P...,Syringe Equipment,Design/Component Flaws
13194,3000209941,Inter-Med Llc,Devices,Class II,Terminated,"US, South Africa, Switzerland, Poland, Italy, ...",Racine,Wisconsin,United States,10/01/2019,The affected lots may be packaged in syringes ...,"Vista Dental Products 3% Sodium Hypochlorite, ...",83435,Class II,175281,CDRH,https://www.accessdata.fda.gov/scripts/ires/?P...,Syringe Equipment,Design/Component Flaws
13195,3000209941,Inter-Med Llc,Devices,Class II,Terminated,"US, South Africa, Switzerland, Poland, Italy, ...",Racine,Wisconsin,United States,10/01/2019,The affected lots may be packaged in syringes ...,Vista Dental Products Chlor-XTRA - 3mL Prefill...,83435,Class II,175287,CDRH,https://www.accessdata.fda.gov/scripts/ires/?P...,Syringe Equipment,Design/Component Flaws
13196,3000209941,Inter-Med Llc,Devices,Class II,Terminated,"US, South Africa, Switzerland, Poland, Italy, ...",Racine,Wisconsin,United States,10/01/2019,The affected lots may be packaged in syringes ...,Vista Dental Products Chlor-XTRA - 12mL Prefil...,83435,Class II,175288,CDRH,https://www.accessdata.fda.gov/scripts/ires/?P...,Syringe Equipment,Design/Component Flaws


In [ ]:
# 1. select a filter
#filtered_df = main_df[(main_df["Device Type"] == 'Cardiac Assist Devices') & (main_df["Recall Category"] == 'Manufacturing Defects')]
filtered_df = main_df[(main_df["Device Type"] == 'COVID-19 Testing Kits')]
# 2. only keep unique Reason for Recall values
unique_df = filtered_df.drop_duplicates(subset='Reason for Recall')

# Summary Information
rows = len(unique_df)  # Number of unique devices found (rows)
list_dev_types = unique_df['Device Type'].unique()  # list of unique device types
list_rec_categories = unique_df['Recall Category'].unique()  


print(f'''
      Summary:
      
      Number of unique devices found: {rows}
      Unique device types: {list_dev_types}
      Unique recall categories: {list_rec_categories}
''')

unique_df

### Graphs !

In [ ]:
import plotly.express as px


filtered_df = main_df[(main_df["Device Type"] == 'Cardiac Assist Devices')]

# Group by 'Recall Category' and count occurrences
recall_counts = filtered_df["Recall Category"].value_counts().reset_index()
recall_counts.columns = ['Recall Category', 'Count']

# Create a bar chart using Plotly
fig = px.bar(recall_counts, x='Recall Category', y='Count', title='Counts of Recall Categories for Cardiac Assist Devices')

# Update layout to rotate x-axis labels
fig.update_layout(xaxis_title='Recall Category', yaxis_title='Count', xaxis_tickangle=-45)

fig


In [217]:
filtered_df = main_df[(main_df["Recall Category"] == 'Manufacturing Defects')]

# Group by 'Device Type' and count occurrences
recall_counts = filtered_df["Device Type"].value_counts().reset_index()
recall_counts.columns = ['Device Type', 'Count']

# Create a bar chart using Plotly
fig = px.bar(recall_counts, x='Device Type', y='Count', title='Counts of Device Types for Manufacturing Defects')

# Update layout to rotate x-axis labels
fig.update_layout(xaxis_title='Device Types', yaxis_title='Count', xaxis_tickangle=-45)

fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'Device Type=%{x}<br>Count=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['Catheters', 'Anesthesia Equipment', 'Blood Pumps', 'Other',
                          'Trachea Devices', 'Ventilation and Respiratory Devices',
                          'Infusion Pumps', 'Cardiac Assist Devices', 'Imaging/Radiology Systems',
                          'Syringe Equipment', 'Surgery Procedure Kits', 'Monitoring Devices',
                          'Craniotomy Equipment', 'COVID-19 Testing Kits'], dtype=object),
              'xaxis': 'x',
              'y': array([146,  50,  46,  38,  31,  28,  20,  19,  11,   6,   4,   3,   2,   1]),
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Counts of Device Types for Manufacturing Defects'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'tickangle': -45, 'title': {'text': 'Device Types'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Count'}}}
})

## Lemmatization

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
df = pd.DataFrame(pd.read_csv("devices_food_drug_raw.csv"))
df['Reason for Recall'] = df['Reason for Recall'].str.lower().fillna('')

def lemmatize_text(text):
    return ' '.join([token.lemma_ for token in nlp(text) if not token.is_punct and not token.is_space])

df['Lemmatized Reason'] = df['Reason for Recall'].apply(lemmatize_text)

df.to_csv('lemmatized.csv', index=False)

In [26]:
df = pd.DataFrame(pd.read_csv("lemmatized.csv"))

# Remove day from date data
df['Center Classification Date'] = pd.to_datetime(df['Center Classification Date'], errors='coerce')
df['Year-Month'] = df['Center Classification Date'].dt.to_period('M').astype(str)  # Group by year and month
df.drop(columns=['Center Classification Date'], inplace=True)

df.to_csv('lemmatized1.csv', index=False)


/var/folders/4n/lg_9zm91673b9xcs269mpwg00000gp/T/ipykernel_59432/3492181976.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.DataFrame(pd.read_csv("lemmatized.csv"))


### Visualizations